In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *   #lit,col,expr,regexp_extract,regexp_replace,current_date, current_timestamp,date_format
from pyspark.sql.types import StructField,StringType,StructType,DoubleType,LongType,IntegerType,FloatType,TimestampType,DataType
import pyspark

In [2]:
spark= SparkSession.builder.appName("DataFrame").getOrCreate()

In [3]:
df=spark.read.csv("C:\\Users\\Steven\\Downloads\\C_DIGO__NICO_DE_MEDICAMENTOS_OTROS_ESTADOS.csv",header=True,inferSchema=True)

In [7]:
# Con esta funcion aplico transformaciones en las columnas 
df=df.withColumn("consecutivocum",col("consecutivocum").cast(IntegerType()))\
    .withColumn("cantidadcum",col("cantidadcum").cast(IntegerType()))\
    .withColumn("expedientecum",col("expedientecum").cast(FloatType()))\
    .withColumn("cantidadcum",col("cantidadcum").cast(IntegerType()))\
    .withColumn("expedientecum",col("expedientecum").cast(FloatType()))\
    .withColumn("fechaactivo",regexp_extract(col("fechaactivo"),"(\d{4}-\d{2}-\d{2})",1))

In [9]:
df.cache()
df.createOrReplaceTempView("dfTabla")

In [10]:
#Contar todas las filas

spark.sql("SELECT count(*) FROM dfTabla").show()
df.select(count(col("titular"))).show()

+--------+
|count(1)|
+--------+
|   79118|
+--------+

+--------------+
|count(titular)|
+--------------+
|         79118|
+--------------+



In [20]:
#Mostrar distict 
spark.sql("SELECT distinct(titular) FROM dfTabla").show(10,False)
df.select(countDistinct(col("titular"))).show(5,10)

+------------------------------------------------------------+
|titular                                                     |
+------------------------------------------------------------+
|LEIRAS OY PHARMACEUTICALS                                   |
|BRISTOL MYERS SQUIBB DE COLOMBIA LTDA                       |
|GENERICOS DE COLOMBIA LTDA                                  |
|BAXTER HEALTHCARE CORPORATION                               |
|COMERCIALIZADORA INTERNACIONAL SOLOGENERICOS S.A.S          |
|INDUSTRIA FARMACEUTICA SERONI S.A.                          |
|ANZG LTDA                                                   |
|INVESTIGACIONES Y DESARROLLO FARMOQUIMICO LTDA (IDEFAR LTDA)|
|CRISTALIA PRODUCTOS QUÍMICOS FARMACÉUTICOS LTDA             |
|NYCOMED IRELAND LTD                                         |
+------------------------------------------------------------+
only showing top 10 rows

+-----------------------+
|count(DISTINCT titular)|
+-----------------------+
|             

In [25]:
#Mostrar distict 
spark.sql("SELECT titular FROM dfTabla GROUP BY titular").show(10,False)

+------------------------------------------------------------+
|titular                                                     |
+------------------------------------------------------------+
|LEIRAS OY PHARMACEUTICALS                                   |
|BRISTOL MYERS SQUIBB DE COLOMBIA LTDA                       |
|GENERICOS DE COLOMBIA LTDA                                  |
|BAXTER HEALTHCARE CORPORATION                               |
|COMERCIALIZADORA INTERNACIONAL SOLOGENERICOS S.A.S          |
|INDUSTRIA FARMACEUTICA SERONI S.A.                          |
|ANZG LTDA                                                   |
|INVESTIGACIONES Y DESARROLLO FARMOQUIMICO LTDA (IDEFAR LTDA)|
|CRISTALIA PRODUCTOS QUÍMICOS FARMACÉUTICOS LTDA             |
|NYCOMED IRELAND LTD                                         |
+------------------------------------------------------------+
only showing top 10 rows



In [5]:
spark.sql("SELECT expediente, count(expediente) as total FROM dfTabla GROUP BY expediente").show(2,False)
df.groupBy(col("expediente")).agg(count(col("expediente").alias("Total"))).show(2)

+----------+-----+
|expediente|total|
+----------+-----+
|58509     |8    |
|19908309  |8    |
+----------+-----+
only showing top 2 rows

+----------+--------------------------+
|expediente|count(expediente AS Total)|
+----------+--------------------------+
|     58509|                         8|
|  19908309|                         8|
+----------+--------------------------+
only showing top 2 rows



# Select importante 


In [27]:
# Paro en primera instacia un select para luego un selectexp hacer muchos calculos

df.select(sum(col("cantidadcum")).alias("Cantidad"),count(col("cantidadcum")).alias("Total"),avg("cantidadcum").alias("avg")).selectExpr("Cantidad/Total","avg").show()

+------------------+-----------------+
|(Cantidad / Total)|              avg|
+------------------+-----------------+
| 47.01087472268175|47.01087472268175|
+------------------+-----------------+



In [30]:
df.agg(collect_set("cantidadcum"),collect_list("cantidadcum")).show(1,False)   #Set listados unicos list todos

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
df.groupBy("expediente").pivot("concentracion").sum("cantidadcum").show()   #Generar otro tipo de tablas

+----------+----+----+----+----+----+----+----+
|expediente|   A|   B|   C|   D|   E|   F|   S|
+----------+----+----+----+----+----+----+----+
|  19942187|null| 540|null|null|null|null|null|
|  19972902| 288|null|null|null|null|null|null|
|  19929669| 248|null|null|null|null|null|null|
|  19977883| 120|null|null|null|null|null|null|
|  19968463| 100|null|null|null|null|null|null|
|     31703|null|null|null|1914|null|null|null|
|  19972247|1410|null|null|null|null|null|null|
|  19908309| 186|null|null|null|null|null|null|
|  19981792|null| 480|null|null|null|null|null|
|  19980027| 888|null|null|null|null|null|null|
|  20003470|null|null|null|null|  12|null|null|
|  19970539|null|  34|null|null|null|null|null|
|  19979478|1800|null|null|null|null|null|null|
|     39527|null|null|null| 126|null|null|null|
|  19981258| 184|null|null|null|null|null|null|
|  19926179|null|null|null|null|null|null| 224|
|  19994619| 162|null|null|null|null|null|null|
|  19903731|  42|null|null|null|null|nul